In [1]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import os
import time
from collections import defaultdict
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,roc_curve
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
import random
random.seed(1)
from sklearn.linear_model import Lasso

/Applications/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


* The symptoms included are as follows:

In [2]:
symptoms = ['intercept',
            'fever',
            'sorethroat',
            'cough',
            'muscle',
            'virus']
aucs_ = defaultdict()

In [3]:
print(symptoms)

['intercept', 'fever', 'sorethroat', 'cough', 'muscle', 'virus']


In [4]:
def read_file(filename):
    data = pd.read_csv(filename)
    data['intercept'] = 1
    columns = list(data.columns)
    columns = columns[-1:] + columns[:-1]
    data = data[columns]
#     train_data = data.drop(['virus'],axis =1).as_matrix()
    return data

In [5]:
def read_parameters(filename):
    parameters = pd.read_csv(filename)
    return parameters

#### Get the parameters for the different dataset combinations

In [6]:
directory_ = "./Parameters_Demographics/"
# with_demographics_ = ['with_demographics_new.csv']
with_demographic_parameters = defaultdict()

In [7]:
def return_parameters(file,parameters_of):
    param = read_parameters(file)
    parameter_dict = defaultdict()
    for i in parameters_of:
        parameter_dict[i] = list(param[i])
    return parameter_dict
    

In [8]:
def get_parameters(dataset_name,parameters):
    return np.array(list(parameters[dataset_name]))

In [9]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [10]:
def get_results(param,sample_points):
    return sigmoid(np.dot(param,sample_points.T)  )

In [11]:
def save_results_for_finding_threshold(filename,dataframe,predicted):
    results = pd.DataFrame()
    results['Actual'] = dataframe['virus']
    results['Predicted'] = predicted
    print(results.head())
    results.to_csv(filename,index = False)

In [12]:
def get_all_datasets(training_data_,training_directory):
    datasets = defaultdict()
    for i in training_data_:
        data = read_file(training_directory+i)
        datasets[i[:-4]] = (data)
    return datasets

In [13]:
def get_all_results(data_dict,param):
    results = defaultdict()
    for i in list(param.keys()):
        data,train = data_dict[i]
        results[i] = get_results(param[i],train)
    return results

In [14]:
def result_statistics(list_):
#     print("Min : ",min(list_))
#     print("Max : ",max(list_))
#     print("Mean : ",np.mean(list_))
#     print("Standard Deviation : ",np.std(list_))
    return min(list_),max(list_)

In [15]:
def return_class(threshold,list_):
    ans = [1 if x >= threshold else 0 for x in list_]
    return ans

def metrics_pred(list1,list2):
    f1 =f1_score(list1,list2)
    precision = precision_score(list1,list2)
    recall = recall_score(list1,list2)
    accuracy = accuracy_score(list1,list2)
    fpr,tpr,threshold = roc_curve(list1,list2)
    auc = metrics.auc(fpr,tpr)
#     print("f1 score : ",f1)
#     print("Precision score : ",precision)
#     print("Recall : ",recall)
#     print("Accuracy : ",accuracy)
#     print("Area under the curve : ",auc)
    return accuracy

In [16]:
def find_threshold(min_,max_,list1,list2,step_size = 1e-3):
    auc_thresholds = defaultdict()
    value = min_
    while value < max_:
        auc_thresholds[value] = metrics_pred(list1,return_class(value,list2))
        value += step_size
    optimal_threshold = max(auc_thresholds.items(), key=lambda x: x[1]) 
    return optimal_threshold

In [17]:
def get_threshold(pred,true):
    min_,max_ = result_statistics(pred)
    threshold = find_threshold(min_,max_,true,pred)
    return threshold

In [18]:
def return_all_thresholds(results,data,y_true):
    thresholds = defaultdict()
    for i in list(data.keys()):
        print("_____________________")
        min_,max_ = result_statistics(results[i])
        
        threshold = find_threshold(min_,max_,y_true[i],results[i])
        print("Found threshold for : ",i)
        thresholds[i] = threshold
    return thresholds

In [19]:

def test(filename_,param,thresholds_):
    aucs = defaultdict()
    data,train = read_file(filename_)
    for i in list(param.keys()):
        test_results = get_results(param[i],train)
        auc_ = metrics_pred(data['virus'],return_class(thresholds_[i][0],test_results))
        aucs[i] = auc_
    return aucs

In [20]:
def return_final_auc_scores(training_data_,training_directory,filename_,parameters):
    data = get_all_datasets(training_data_)
    results = get_all_results(data,parameters)
    #find the thresholds
    thresholds = return_all_thresholds(results,data)
    #get the auc values
    aucs_= test(filename_,parameters,thresholds)
    return aucs_


In [21]:
def create_dict(dict_):
    temp = []
    for k,v in dict_.items():
        temp.append((k,v))
    return temp
        

In [22]:
results_symp = defaultdict()
results_demo = defaultdict()

#### Get the symptoms

In [23]:
def get_gender(dataframe_):
    df = dataframe_[['male','female']]
    temp = df.apply(lambda x:x.argmax(),axis =1)
    return temp

In [24]:
def get_age(dataframe_):
    df = dataframe_[['age 0-4', 'age 5-15', 'age 16-44', 'age 45-64', 'age 65+']]
    temp = df.apply(lambda x: x.argmax(), axis=1)
    return temp

In [81]:
def get_predictions_all(name,train,only_symp_age,only_symp_gender,param_dict,temp_age,temp_gender,collection_mode = 'clinically_collected',population ='population'):
    results = []
    for i in range(train.shape[0]):
        result = []
        sample_point = train[i,:]
        gender = list(only_symp_gender.iloc[i][:])
        age = list(only_symp_age.iloc[i][:])
        p_data = get_results(param_dict[name],sample_point)
        result.append(p_data)
#         result.append(gender[0]*get_results(param_dict['male'],sample_point))
#         result.append(gender[1]*get_results(param_dict['female'],sample_point))

#         result.append(age[0]*get_results(param_dict['age 0-4'],sample_point))
#         result.append(age[1]*get_results(param_dict['age 5-15'],sample_point))
#         result.append(age[2]*get_results(param_dict['age 16-44'],sample_point))
#         result.append(age[3]*get_results(param_dict['age 45-64'],sample_point))
#         result.append(age[4]*get_results(param_dict['age 65+'],sample_point))
        
#         p_collection = get_results(param_dict[collection_mode],sample_point)
#         p_gender = get_results(param_dict[temp_gender[i]],sample_point)
#         p_age = get_results(param_dict[temp_age[i]],sample_point)
# #         p_population = get_results(param_dict[population],sample_point)
#         result = [p_data,p_collection,p_gender+p_age]
        results.append(result)
    return results
    
    

In [82]:
def get_coeff(X,Y):
    lm = linear_model.LogisticRegression()
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
    lm.fit(x_train,y_train)
    y_pred = lm.predict(x_test)
    acc = accuracy_score(y_test,y_pred)
#     print("Accuracy :",acc)
    fpr,tpr,threshold = roc_curve(y_test,y_pred)
    auc_score = metrics.auc(fpr,tpr)
#     print("AUC :",auc_score)
    coefficients = lm.coef_.tolist()[0]
    print("Coefficients : ",coefficients)
    intercept = lm.intercept_.tolist()[0]
    return coefficients,intercept
    

In [83]:
def norm(list_):
    min_ = min(list_)
    max_ = max(list_)
    denom = max_ - min_
    ans = [x-min_/denom for x in list_]
    return ans

In [84]:
COLLECTION_MODE = {'nyumc':'clinically_collected',
                   'goviral':'individually_reported',
                   'fluwatch':'individually_reported',
                   'hongkong': 'health_worker',
                   'hutterite':'health_worker','loeb':'healt_worker'}

In [85]:
def process_all(training_data_list,training_directory,filename_,parameters,collection_mode = COLLECTION_MODE):
    name_dataset = filename_.split('/')[-1]
    thresholds = defaultdict()
    print(name_dataset)
    data = get_all_datasets(training_data_list,training_directory)
    print("Got the data")
    print("Now finding coefficients for the the datasets!")
    weights = defaultdict()
    for i in data.keys():
        print("Analyzing the dataset : ",i)
        data_ = data[i]
        temp_age = get_age(data_)
        temp_gender = get_gender(data_)
        only_symp_data = data_[symptoms]
        only_symp_gender = data_[['male','female']]
        only_symp_age = data_[['age 0-4','age 5-15','age 16-44','age 45-64','age 65+']]
        only_symp_data.drop('virus',axis = 1,inplace = True)
        train_data_symp = only_symp_data.as_matrix()
        prediction = get_predictions_all(i,train_data_symp,only_symp_age,only_symp_gender,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
        y_true = list(data_['virus'])
        coefficient,intercept = get_coeff(prediction,y_true)
        weights[i] = (coefficient,intercept)
        value = np.array(np.dot(prediction,np.array(weights[i][0]).T)+weights[i][1])
        values = [sigmoid(j) for j in value]
        
        
        threshold = get_threshold(values,y_true)
        print("Found threshold for ",i)
        thresholds[i] = threshold[0]
        ans = [(y_true[i],values[i]) for i in range(len(y_true))]
    return weights,thresholds,ans

In [86]:
def process_test(training_directory,filename_,parameters,weights,thresholds,collection_mode = COLLECTION_MODE):
    aucs_ = defaultdict()
    predictions = defaultdict()
    test_data = get_all_datasets([filename_],training_directory)
    name = filename_.split('.')[0]
    print("Name : ",name) 
    data_ = test_data[name]
    temp_age = get_age(data_)
    temp_gender = get_gender(data_)
    only_symp_data = data_[symptoms]
    only_symp_data.drop('virus',axis = 1,inplace = True)
    only_symp_gender = data_[['male','female']]
    only_symp_age = data_[['age 0-4','age 5-15','age 16-44','age 45-64','age 65+']]
    y_true = list(data_['virus'])
    train_data_symp = only_symp_data.as_matrix()
    for i in weights.keys():
        print("Using the parameters of : ",i)
        prediction = get_predictions_all(i,train_data_symp,only_symp_age,only_symp_gender,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
#         temp = [i[1:] for i in prediction]
#         first = [i[0] for i in prediction]
        prediction = np.array(prediction)
        value = np.array(np.dot(prediction,np.array(weights[i][0]).T)+weights[i][1])
        values = [sigmoid(i) for i in value]
        predictions[i] = values
   
    print("Got the predicitions from the different parameters")
    for i in weights.keys():
        auc_ = metrics_pred(y_true,return_class(thresholds[i],predictions[i]))
        aucs_[i] = auc_
        print("Found the auc for ",i)
    return aucs_
#         

#### With demographics

##### Generating the results for NYUMC

In [87]:
training_data_nyumc = ['goviral.csv']
training_directory = "../../../../Sampled_data/Goviral/Train/"
testing_directory = "../../../../Sampled_data/Goviral/Test/"
filename_ = 'goviral.csv'

In [88]:
cols = ['goviral', 'male','female', 'population']
demo_nyumc = return_parameters(directory_+'with_demographics_goviral.csv',cols)
demo_nyumc.keys()

dict_keys(['male', 'goviral', 'female', 'population'])

In [89]:
print("With demographics!")

With demographics!


In [90]:
weights_nyumc,thresholds_nyumc,pred = process_all(training_data_nyumc,training_directory,filename_,demo_nyumc)

goviral.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  goviral
Coefficients :  [1.6110053061304248]
Found threshold for  goviral


In [91]:
thresholds_nyumc

defaultdict(None, {'goviral': 0.5906957378448827})

In [92]:
weights_nyumc

defaultdict(None, {'goviral': ([1.6110053061304248], -0.7843374106747217)})

In [93]:
aucs_nyumc = process_test(testing_directory,filename_,demo_nyumc,weights_nyumc,thresholds_nyumc)

Name :  goviral
Using the parameters of :  goviral
Got the predicitions from the different parameters
Found the auc for  goviral


In [94]:
aucs_nyumc

defaultdict(None, {'goviral': 0.7235576923076923})

In [95]:
aucs_['goviral'] = aucs_nyumc

##### Generating the results for FluWatch


##### Generating the results for hongkong

In [96]:
training_data_fluwatch = ['hongkong.csv']
# training_directory = "../../Data/With_Improved_Target/With_Demographics/"
training_directory = "../../../../Sampled_data/Hongkong/Train/"
testing_directory = "../../../../Sampled_data/Hongkong/Test/"
filename_ = 'hongkong.csv'

In [97]:
cols = ['hongkong','male','female', 'population']
demo_fluwatch = return_parameters(directory_+'with_demographics_hongkong.csv',cols)
demo_fluwatch.keys()

dict_keys(['male', 'hongkong', 'female', 'population'])

In [98]:
weights_fluwatch,thresholds_fluwatch,ans = process_all(training_data_fluwatch,training_directory,filename_,demo_fluwatch)

hongkong.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  hongkong
Coefficients :  [8.486185384941479]
Found threshold for  hongkong


In [99]:
aucs_fluwatch1 = process_test(testing_directory,filename_,demo_fluwatch,weights_fluwatch,thresholds_fluwatch)

Name :  hongkong
Using the parameters of :  hongkong
Got the predicitions from the different parameters
Found the auc for  hongkong


In [100]:
aucs_fluwatch1

defaultdict(None, {'hongkong': 0.9149848637739657})

In [101]:
aucs_['hongkong'] = aucs_fluwatch1

##### Generating the results for Hutterite

In [102]:
training_data_hongkong = ['hutterite.csv']
# training_directory = "../../Data/With_Improved_Target/With_Demographics/"
training_directory = "../../../../Sampled_data/Hutterite/Train/"
testing_directory = "../../../../Sampled_data/Hutterite/Test/"
filename_ = 'hutterite.csv'

In [103]:
cols = ['hutterite',  'male','female', 'population']
demo_hongkong = return_parameters(directory_+'with_demographics_hutterite.csv',cols)
demo_hongkong.keys()

dict_keys(['hutterite', 'population', 'male', 'female'])

In [104]:
weights_hongkong,thresholds_hongkong,ans = process_all(training_data_hongkong,training_directory,filename_,demo_hongkong)

hutterite.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  hutterite
Coefficients :  [2.8665140106850666]
Found threshold for  hutterite


In [105]:
aucs_hongkong = process_test(testing_directory,filename_,demo_hongkong,weights_hongkong,thresholds_hongkong)

Name :  hutterite
Using the parameters of :  hutterite
Got the predicitions from the different parameters
Found the auc for  hutterite


In [106]:
aucs_hongkong

defaultdict(None, {'hutterite': 0.7219512195121951})

In [107]:
aucs_['hutterite'] = aucs_hongkong

In [108]:
aucs_

defaultdict(None,
            {'goviral': defaultdict(None, {'goviral': 0.7235576923076923}),
             'hongkong': defaultdict(None, {'hongkong': 0.9149848637739657}),
             'hutterite': defaultdict(None,
                         {'hutterite': 0.7219512195121951})})